In [5]:
import numpy as np
import cv2

# Image

In [3]:
face_cas = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

img = cv2.imread('ff.jpg')
resize_img = cv2.resize(img,(640,480))
gray_img = cv2.cvtColor(resize_img, cv2.COLOR_BGR2GRAY)

faces = face_cas.detectMultiScale(gray_img, 1.029, 5)

for (x, y, w, h) in faces:
    cv2.rectangle(resize_img, (x, y), (x + w, y + h), (255, 0, 0), 2)

cv2.imshow('img', resize_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Video

In [5]:
face_cas = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    resized_frame = cv2.resize(frame, (1200, 800))
    gray_img = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    faces = face_cas.detectMultiScale(gray_img, scaleFactor= 1.1, minNeighbors= 5, minSize= (300, 300))

    for (x, y, w, h) in faces:
        cv2.rectangle(resized_frame, (x, y), (x + w, y + h), (0, 255, 255), 10)

    cv2.imshow('video', resized_frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

 # Data Preparation

In [7]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.8.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.23.0%2Bcu126-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached https://download.pytorch.org/whl/cu126/torch-2.8.0%2Bcu126-cp311-cp311-win_amd64.whl (2915.5 MB)
Using ca


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import math
import torch.optim as optim
from torchvision import transforms, datasets

In [9]:
torch.__version__

'2.8.0+cu126'

In [3]:
Data_dir = "FER_2013"
BATCH_SIZE = 64
EPOCHS = 40
LR = 1e-3
MODEL_OUT = "emotion_model.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CLASS_MAP = ['angry','disgust','fear','happy','sad','surprise','neutral']
NUM_CLASSES = len(CLASS_MAP)

# --- Transforms ---
transform = transforms.Compose([
    transforms.Grayscale(),   # ensure single channel
    transforms.Resize((48,48)),
    transforms.ToTensor(),    # scales to [0,1]
])

# --- Dataset ---
train_ds = datasets.ImageFolder(root=f"{Data_dir}/train", transform=transform)
val_ds   = datasets.ImageFolder(root=f"{Data_dir}/test", transform=transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

# --- Model ---
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # compute flattened size automatically
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 48, 48)  # 1 grayscale image
            dummy_out = self._forward_conv(dummy)
            flat_size = dummy_out.view(1, -1).size(1)

        self.fc1 = nn.Linear(flat_size, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def _forward_conv(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)

# --- Train ---
def train_model():
    model = EmotionCNN().to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, pred = out.max(1)
            correct += pred.eq(y).sum().item()
            total += y.size(0)
        train_acc = correct / total

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                out = model(X)
                _, pred = out.max(1)
                correct += pred.eq(y).sum().item()
                total += y.size(0)
        val_acc = correct / total

        print(f"Epoch {epoch+1}/{EPOCHS} - Loss {total_loss/len(train_loader):.4f} - Train acc {train_acc:.3f} - Val acc {val_acc:.3f}")

    torch.save(model.state_dict(), MODEL_OUT)
    print("Model saved:", MODEL_OUT)


In [ ]:
if __name__ == "__main__":
    train_model()

# Load Model

In [4]:
import cv2
import torch
import torch.nn.functional as F
import numpy as np

# Make sure IMG_SIZE matches training
IMG_SIZE = 48
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to model and class map
MODEL_OUT = "emotion_model.pth"
CLASS_MAP = ['angry','disgust','fear','happy','sad','surprise','neutral']

# Load face cascade properly
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def preprocess_face(face_gray):
    face = cv2.resize(face_gray, (IMG_SIZE, IMG_SIZE))
    face = face.astype(np.float32) / 255.0
    face = np.expand_dims(face, axis=0)  # channel
    face = np.expand_dims(face, axis=0)  # batch
    return torch.tensor(face, dtype=torch.float32).to(DEVICE)

def main(video_source=0):
    # Load model
    model = EmotionCNN()
    model.load_state_dict(torch.load(MODEL_OUT, map_location=DEVICE))
    model.to(DEVICE).eval()

    cap = cv2.VideoCapture(video_source)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

        for (x, y, w, h) in faces:
            face_gray = gray[y:y+h, x:x+w]
            inp = preprocess_face(face_gray)

            with torch.no_grad():
                out = model(inp)
                probs = F.softmax(out, dim=1).cpu().numpy()[0]
                class_id = int(np.argmax(probs))
                label = CLASS_MAP[class_id]
                prob = probs[class_id]

            text = f"{label} {prob:.2f}"
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(frame, text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        cv2.imshow("PyTorch Emotion Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(0)


# Adding a layer

In [10]:
Data_dir = "FER_2013"
BATCH_SIZE = 64
EPOCHS = 300
LR = 1e-3
MODEL_OUT = "emotion_model2.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CLASS_MAP = ['angry','disgust','fear','happy','sad','surprise','neutral']
NUM_CLASSES = len(CLASS_MAP)

# --- Transforms ---
transform = transforms.Compose([
    transforms.Grayscale(),   # ensure single channel
    transforms.Resize((48,48)),
    transforms.ToTensor(),    # scales to [0,1]
])

# --- Dataset ---
train_ds = datasets.ImageFolder(root=f"{Data_dir}/train", transform=transform)
val_ds   = datasets.ImageFolder(root=f"{Data_dir}/test", transform=transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

# --- Model ---
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout_conv = nn.Dropout2d(0.25)
        self.dropout_fc = nn.Dropout(0.5)

        # compute flattened size automatically
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 48, 48)  # 1 grayscale image
            dummy_out = self._forward_conv(dummy)
            flat_size = dummy_out.view(1, -1).size(1)

        self.fc1 = nn.Linear(flat_size, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def _forward_conv(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout_conv(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout_conv(x)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)
        x = self.dropout_fc(F.relu(self.fc1(x)))
        return self.fc2(x)

# --- Train ---
def train_model():
    model = EmotionCNN().to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            out = model(X)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, pred = out.max(1)
            correct += pred.eq(y).sum().item()
            total += y.size(0)
        train_acc = correct / total

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                out = model(X)
                _, pred = out.max(1)
                correct += pred.eq(y).sum().item()
                total += y.size(0)
        val_acc = correct / total

        print(f"Epoch {epoch+1}/{EPOCHS} - Loss {total_loss/len(train_loader):.4f} - Train acc {train_acc:.3f} - Val acc {val_acc:.3f}")

    torch.save(model.state_dict(), MODEL_OUT)
    print("Model saved:", MODEL_OUT)


In [7]:

if __name__ == "__main__":
    train_model()

Epoch 1/300 - Loss 2.0463 - Train acc 0.225 - Val acc 0.225
Epoch 2/300 - Loss 1.7933 - Train acc 0.254 - Val acc 0.275
Epoch 3/300 - Loss 1.7390 - Train acc 0.277 - Val acc 0.342
Epoch 4/300 - Loss 1.6908 - Train acc 0.298 - Val acc 0.387
Epoch 5/300 - Loss 1.6592 - Train acc 0.312 - Val acc 0.395
Epoch 6/300 - Loss 1.6425 - Train acc 0.318 - Val acc 0.402
Epoch 7/300 - Loss 1.6171 - Train acc 0.333 - Val acc 0.419
Epoch 8/300 - Loss 1.6151 - Train acc 0.333 - Val acc 0.432
Epoch 9/300 - Loss 1.6027 - Train acc 0.337 - Val acc 0.425
Epoch 10/300 - Loss 1.5967 - Train acc 0.340 - Val acc 0.433
Epoch 11/300 - Loss 1.5812 - Train acc 0.347 - Val acc 0.430
Epoch 12/300 - Loss 1.5808 - Train acc 0.347 - Val acc 0.422
Epoch 13/300 - Loss 1.5709 - Train acc 0.351 - Val acc 0.417
Epoch 14/300 - Loss 1.5562 - Train acc 0.357 - Val acc 0.432
Epoch 15/300 - Loss 1.5556 - Train acc 0.351 - Val acc 0.425
Epoch 16/300 - Loss 1.5437 - Train acc 0.359 - Val acc 0.446
Epoch 17/300 - Loss 1.5434 - Trai

# Load model

In [11]:
import cv2
import torch
import torch.nn.functional as F
import numpy as np

# Make sure IMG_SIZE matches training
IMG_SIZE = 48
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to model and class map
MODEL_OUT = "emotion_model2.pth"
CLASS_MAP = ['angry','disgust','fear','happy','sad','surprise','neutral']

# Load face cascade properly
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def preprocess_face(face_gray):
    face = cv2.resize(face_gray, (IMG_SIZE, IMG_SIZE))
    face = face.astype(np.float32) / 255.0
    face = np.expand_dims(face, axis=0)  # channel
    face = np.expand_dims(face, axis=0)  # batch
    return torch.tensor(face, dtype=torch.float32).to(DEVICE)

def main(video_source=0):
    # Load model
    model = EmotionCNN()
    model.load_state_dict(torch.load(MODEL_OUT, map_location=DEVICE))
    model.to(DEVICE).eval()

    cap = cv2.VideoCapture(video_source)

    while True:
        ret, frame = cap.read()
        #if not ret:
            #break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_gray = gray[y:y+h, x:x+w]
            inp = preprocess_face(face_gray)

            with torch.no_grad():
                out = model(inp)
                probs = F.softmax(out, dim=1).cpu().numpy()[0]
                class_id = int(np.argmax(probs))
                label = CLASS_MAP[class_id]
                prob = probs[class_id]

            text = f"{label} {prob:.2f}"
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(frame, text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        cv2.imshow("PyTorch Emotion Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main("vid1.mp4")
